In the winter of 2023, Jordan Tigani declared that [big data is dead](https://motherduck.com/blog/big-data-is-dead/). In this blog post, he makes several claims about how much data that businesses typically query. It is easy to interpret this to mean that big data was just a hype and most of us shouldn't need to concern ourselves with it. I don't think that's the intended interpretation and I think that would be a mistake. The big data hype has lead to a lot of exciting innovations that increases the capability of commodity machines. I think it's unwise to ignore that, although I completely agree that most organizations do not need to set up multi-petabyte data platforms.

I can now easily run analytical queries on my laptop that people might've offloaded to clusters 5 or 10 years ago. Innovations in storage formats makes it possible for me to put quite large data sets on my laptop without having to delete everything else on the disk. NVMe disks are bonkers fast and make it much less punishing to work with data sets that do not fit in RAM. Our analytical tools can use HTTP range requests to only touch the required parts of data sets stored in cheap object storage and retrieve them with blazing speed if the bandwidth is good. Initiatives like [Apache Arrow](https://arrow.apache.org/) makes it cheap to transfer data between different analytics or storage engines on a single machine.

Many more data sets are now easily approachable with a normal laptop than 5 or 10 years ago. I want to write about and demo a technology I've been following for some years now. [DuckDB](https://duckdb.org/) is an in-process database, like sqlite, but tuned for analytical queries, rather than transactional queries. It has bindings for many languages, like Python, Java, Node, Rust and Wasm. It also has its own storage format, which is portable and backwards compatible on a timescale of some years. So just like with sqlite, you can ship it to share your analysis with anyone else.

For me, these are taking over a lot of the things I would've previously used spark for. Like spark, it can run queries on data that is much bigger than RAM. Unlike spark, it's not a clustered solution, so there's much less overhead. It is easy to read data from cheap block storage like S3, RDBMs systems or common storage formats with it, and it's very, very fast. I'm running this on a MacBook Pro M1 Max with 32GB RAM. It would cost me quite a bit of money to run a machine like that in the cloud. If I don't need the horizontal scaling of a cluster, why would I want to insert the overhead of a cluster into my feedback loop?

Today I'm going to do some hobby-analysis of public transportation delays in Trøndelag County to try to get a little familiar with DuckDB. I will be using the [Entur Real-Time](https://data.entur.no/domain/public-transport-data/product/realtime_siri_et/urn:li:container:1d391ef93913233c516cbadfb190dc65) data set of Norwegian public transport. This is a BigQuery table currently containing 2 236 636 725 records of 25 columns, and it claims to be about 500GB. This is definitely within the realm of what I would have considered Big Data not _that_ many years ago. With modern tools, I think it's more in the realm of "inconveniently sized" than "big".

It's possible to download slices of this data set from a [CSV exporter](https://siri-data-exporter.entur.no/), but the CSV files will be very big, and they'll also be [CSV](https://kaveland.no/posts/2024-03-10-friends-dont-let-friends-export-csv/). I will read the data from the BigQuery table instead. I'm primarily interested in transports in Trøndelag, so I will be filtering the data in my initial import, so we'll end up with a significantly smaller subset of this data locally.

I've done some setup to prepare:

1. I made a Google Cloud account. Over at the [google cloud console](https://console.cloud.google.com/) I created a new project. I needed to enable billing to use BigQuery, but this data set is so small that it copying it probably won't incur any actual billing (this tells me that I'm probably not the target audience for BQ).
2. I enabled BigQuery API under APIs & Service somewhere.
3. Under IAM & Admin, I clicked Service Accounts and created one. I gave it the BigQuery User role.
4. On the service account details, I went to Keys and created a new key. This gave me a JSON file that I placed in `~/.bq_credentials`. When I want to authenticate to BigQuery, I make sure I have an environment variable named `GOOGLE_APPLICATION_CREDENTIALS` that points to that file, and the BQ sdk will pick it up.
5. I made the repository that this notebook is in and set it up with `uv init --name bus-eta --no-package --vcs git -p 3.12`.
6. I added the packages I'd like to use with `uv add duckdb pandas google-cloud-bigquery-storage google-cloud-bigquery pyarrow tqdm seaborn jupyter jupysql db-dtypes`. Note that `google-cloud-bigquery-storage` enables more efficient data transfer, using `arrow` or `avro` formats (we'll get arrow since we installed `pyarrow` but not any `avro` package).
7. I added `*.db`, `*.pq`, `*.parquet` to `.gitignore` so I wouldn't accidentally commit a huge data file.
8. If you want to play along, you can clone this repository, export a `GOOGLE_APPLICATION_CREDENTIALS` variable, run `uv sync` and `uv run jupyter lab` to get an interactive copy of this notebook to play around in. You can get uv from [here](https://docs.astral.sh/uv/). VSCode and JetBrains have plugins for working with jupyter notebook that are a lot better than running it in the browser, but the browser will work just fine too.

I haven't used DuckDB much before, I'm doing this to learn! I'm familiar with SQL, and DuckDB is based on the postgres dialect, so I should be fine. I'll be doing visualizations using [seaborn](https://seaborn.pydata.org/) which has a really nice API for quickly making beautiful plots of tabular data. This whole post is going to be my stream of thought as I try to get familiar with both DuckDB and the entur real time dataset.

Now let's get ready to fetch some data! Let's set up the BigQuery client and verify that we can access the table first:

In [ ]:
import math
from datetime import timedelta

import numpy as np
import duckdb
import psutil
from matplotlib.pyplot import yscale
from tqdm.notebook import tqdm
from google.cloud import bigquery
import seaborn as sns
# Set up some styling for the visual stuff
sns.set_theme(
    style='whitegrid', rc={
        'figure.figsize': (12, 8),
        'figure.frameon': False,
        'legend.frameon': False
    }
)

source_table = "ent-data-sharing-ext-prd.realtime_siri_et.realtime_siri_et_last_recorded"
client = bigquery.Client()

client.query(
    f"select dataSource, dataSourceName, count(*) as volume from `{source_table}` where operatingDate > '2025-01-01' group by dataSource, dataSourceName"
).to_dataframe().sort_values(by='volume', ascending=False)

I'm not interested in all of these regional services, so I'll settle for the ones that I could conceivably encounter near Trondheim. That probably means these:

- `ATB` - the regional "brand" for Trøndelag
- `BNR` - BaneNOR, probably local and regional trains.
- `SJN` - SJ Nord operate Dovrebanen between Oslo and Trondheim, as well as Nordlandsbanen between Trondheim and Bodø.

Let's set up a connection to a DuckDB instance at `entur.db` first. This will create persistent storage on my disk, if it doesn't exist yet. The "connection" part is somewhat of a misnomer, since there's no server -- just like with sqlite, everything interesting is happening with files and memory here. I've decided to name the table for the raw data `arrivals`. Since I want to be able to rerun this notebook from top to bottom efficiently, I will only create it if it does not exist already.

From the [documentation](https://duckdb.org/docs/guides/performance/how_to_tune_workloads) I discovered that DuckDB can sometimes create too many threads. I'll manually set a thread count to the number of physical cores, leaving 1 for the rest of the system, to avoid overloading it.

In [ ]:
db = duckdb.connect("entur.db")
cores = psutil.cpu_count(logical=False) - 1
db.execute(f"set threads = {cores};")
exists = db.sql("select exists(select 1 from information_schema.tables where table_name = 'arrivals')").fetchone()[0]

The DuckDB API generally does a good job of mimicking existing database APIs in the various languages. The `fetchone()`-method is part of the [DB-API](https://peps.python.org/pep-0249/) spec.

Next up, we'll write the query for the data set we want from Big Query. If I was planning on doing anything serious, I would probably find a better way of constructing the query strings, I'm certain BQ supports bind parameters. I'm not supplying any user input though, so I won't need it, and BQ isn't the topic of this post.

In [ ]:
query = f"""
SELECT
  recordedAtTime,
  lineRef,
  directionRef,
  operatingDate,
  vehicleMode,
  extraJourney,
  journeyCancellation,
  stopPointRef,
  sequenceNr,
  stopPointName,
  originName,
  destinationName,
  extraCall,
  stopCancellation,
  estimated,
  aimedArrivalTime,
  arrivalTime,
  aimedDepartureTime,
  departureTime,
  datedServiceJourneyId
FROM `{source_table}`
WHERE dataSource in ('ATB', 'BNR', 'SJN') -- NB! This selects trains and Trøndelag. Maybe you want something else?
"""

I'm not interested in running this query more than once. It's going to transfer a lot of data over the wire, hopefully in an efficient binary format like Avro or Parquet, but it's still going to take a while. In case the data is too big for RAM, I will naively paginate it by grabbing 14 days worth of data at a time. The BQ sdk probably has a nicer way of doing this, but it's yet again not the topic of this post. This is probably going to take a while, so it's time to get a coffee...

**Important note**: If you're running this notebook, you may want to adjust the `start`-variable to avoid pulling down all the data since the beginning of time. It'll be much faster, and many things have changed in traffic the past 5 or so years anyway.

In [ ]:
if not exists:
    # download it, paged 1 month per page
    rs = client.query(f"""
    SELECT MIN(operatingDate) as min_record,
           MAX(operatingDate) as max_record
    FROM `{source_table}`
    WHERE dataSource in ('ATB', 'BNR', 'SJN') -- NB! This selects trains and Trøndelag. Maybe you want something else?
    """).result()
    row = list(rs)[0]

    start, end = row.get("min_record"), row.get("max_record")
    page_delta = timedelta(days=30)
    page_count = math.ceil(((end - start) / page_delta))
    page_boundaries = [
        start + n * page_delta
        for n in range(page_count + 1) # include last page
    ]
    created = False
    for start, end in tqdm(list(zip(page_boundaries, page_boundaries[1:]))):
        job = client.query(f"{query} AND operatingDate >= '{start.isoformat()}' AND operatingDate < '{end.isoformat()}'")
        batch = job.to_arrow()
        if created:
            db.execute("insert into arrivals select * from batch")
        else:
            db.execute("create table arrivals as select * from batch")
            created = True

db.query("select count(*) as rows_found from arrivals")

The code above may look strange for a few reasons. There's the string-interpolation query which I apologize for. And then there's this section:
```python
        batch = job.to_arrow()
        if created:
            db.execute("insert into arrivals select * from batch")
        else:
            db.execute("create table arrivals as select * from batch")
```

First, we grab the result set from BigQuery as an arrow table. This is a memory layout and wire format for tabular data defined by the [Apache Arrow](https://arrow.apache.org/) project. DuckDB understands this format natively. BigQuery can use the wire format. There's no `batch` table in `entur.db`, what's happening there is that we're using something called a [replacement scan](https://duckdb.org/docs/clients/c/replacement_scans.html). Since DuckDB is in our process memory, it can access `batch` on the C level and query _directly_ against it, no copying necessary. polars can also understand this memory layout, as can pandas, and many other in-process analytical engines, like R for example.

This took a surprisingly long time to run the first time around, around 20 minutes on my 750/750 Mbps line. Since we're selecting only around 1/25th of the data, we could extrapolate that we'd need 8 hours or so to download the whole thing. This may seem like a lot, but considering how "little" extra data is added over time here, an initial slow sync, followed by a daily update to keep the laptop in sync would be totally fine. I'm pretty confident my laptop could handle analytical queries on that amount of data just fine.

In case you haven't noticed yet, this blog is actually a Jupyter Notebook. That enables us to promote DuckDB into a first class citizen by enabling `jupysql`, so let's do that:


In [ ]:
%load_ext sql
%sql db --alias duckdb
%config SqlMagic.displaylimit = 50

Now we can very easily mix and match Python for visualization and SQL for analysis. Let's give it a go:

In [ ]:
rs = %sql SELECT COALESCE(vehicleMode, 'Unknown') as vehicle, COUNT(*) as count FROM ARRIVALS GROUP BY vehicleMode
sns.barplot(rs.DataFrame(), x='vehicle', y='count');

Right, we've got it working. I will prefer to use the `%%sql` syntax that turns a whole cell/codeblock into sql, instead of the `%sql` which only does it on the rest of the line. It's going to make syntax highlighting work out better.

It seems like perhaps `vehicleMode` is a relatively new field, or maybe not exposed by all the transport vendors. We did store all the data to `entur.db`. There are some 200 million rows in there. Let's check how big it is:

In [ ]:
!du -hs entur.db

Just for fun, let's find out what size it would be in CSV (then immediately delete it):

In [ ]:
%time db.execute("COPY arrivals TO 'arrivals.csv' (HEADER, DELIMITER ';')")
!du -hs arrivals.csv
!time wc -l arrivals.csv
!rm arrivals.csv

Quick summary of that, the `entur.db` DuckDB file is roughly 8 times smaller than the 56GB CSV file, which we wasted 50+ seconds writing. It took `wc` even longer than that to scan through the file. I'd be willing to bet real money that DuckDB can do an aggregation across this table in less than a second on my machine.

Let's check the busiest stations. We will use the DuckDB relations API and replacement scans to make this convenient, so we can break it up into multiple queries.

In [ ]:
%%time
by_station_name = db.sql("select stopPointName, count(*) as count from arrivals group by stopPointName")
top_10 = db.sql("select stopPointName, count from by_station_name order by count desc limit 10")
top_10.df()

That was 117ms the first time I ran it, and I didn't have to do anything clever.

Let's check how a `.parquet` file would come out, that too would probably give us a compact file that we could query efficiently:

In [ ]:
%time db.execute("COPY arrivals TO 'arrivals.parquet'")
!du -hs arrivals.parquet


This has even better compression, and DuckDB can easily query this too:

In [ ]:
%%time
by_station_name = db.sql("select stopPointName, count(*) as count from 'arrivals.parquet' group by stopPointName")
top_10 = db.sql("select stopPointName, count from by_station_name order by count desc limit 10")
top_10.df()

That's a bit longer at 1.6s on first recording, but still quick. Compact binary formats are *nice*, we can do very fast aggregations. Both the DuckDB format and `.parquet` files contain schemas, and enough information so that the majority of the file can be skipped. They're also much more compact, so reading into memory from them is also fast. I feel like it's time to start figuring out what the data actually looks like.

## Exploring the data

As far as I can tell, each row is a record of a transport service at a physical location and time, together with information about the planned, estimated and actual schedule. Let's look at one whole example. I have picked a bus stop where I frequently take the bus, and hardcoded `datedServiceJourneyId` for one particular journey. I don't want this notebook to accidentally change if I load more data at some later point. I know this as the "14 bus".

In [ ]:
journey = "2024-02-10:ATB:ServiceJourney:14_230306097868553_6050"

db.sql("""
select * from arrivals
where datedServiceJourneyId = ? and stopPointName = 'Åsvang skole'
limit 1
""", params=[journey]).df().T

Okay, so this the bus service I use for work, it's the only bus to visit this stop point. That helps me get my bearings.

There's a `stopPointRef`, which is probably the ID of the stop point. It's probably a reference to the ID in [this dataset]( I'm going to try looking it up in [this dataset](https://data.entur.no/domain/public-transport-data/product/national_stop_registry/urn:li:container:7fd60886cfa95c660d5c85ac0384eb9a), which I could use to learn more about this stop.

`lineRef` looks like some sort of composite hierarchical ID. The bus has ATB-branding, and the number 14. The 2 is probably a code or ID for something. We'll figure that out later.

There's no `vehicleMode`, but I know that this is a bus. I could probably backfill this column by identifying that this stop point is for buses if I wanted to, but I don't think I need this column for my analysis.

I'm not sure what I would want to do with the `extraJourney` column. This seems likely to be train-related, according to the documentation this means the vehicle journey is a replacement journey (probably like bus for train or bus for ferry). For now, I'll disregard it. If there's a `journeyCancellation`, I think it's unlikely that we'll see a value for `arrivalTime` or `departureTime`, so I guess we don't need that either.

`extraCall` means that the stop at this place was in addition to the scheduled stops. I don't think I need to worry about this. `stopCancellation` means that the service skipped a stop that was planned. I also don't think I need this, nor `estimated`, as I only care about scheduled and actual times.

As far as I can tell, `datedServiceJourneyId` can be used to find all the registrations for one journey, and they should be ordered by their `SequencNr`. Let's check that:

In [ ]:
%%sql

select
  stopPointName,
  aimedArrivalTime,
  arrivalTime,
  aimedDepartureTime,
  departureTime
from arrivals
-- apparently %%sql uses jinja templates
where arrivals.datedServiceJourneyId = '{{ journey }}'
order by sequenceNr asc;

Okay, that looks like it's the correct interpretation, I take this bus quite often. This particular journey, the bus has been delayed according to the planned schedule. That's exactly the kind of thing I'm interested in looking for. I can easily check _how_ delayed it is at any given stop by matching the `aimedArrivalTime` to the `arrivalTime`. This would let me answer questions like what time of the day that frequent delays at any given stop.

Let's pick the highest volume stop and check the mean delay by hour of day, month, and by year.

In [ ]:
df = db.sql("""
select
  mean(extract(epoch from arrivalTime - aimedArrivalTime)) as delay_seconds,
  extract(year from arrivalTime) :: text as year,
  extract(hour from arrivalTime) as hour,
  extract(month from arrivalTime) as month
from arrivals
where stopPointName = 'Nidarosdomen'
group by 2, 3, 4
""").df()

sns.lineplot(
    df, x='hour', hue='year', y='delay_seconds', palette='tab20'
);

From this we can conclude only that the data registration isn't perfect (this is usually what happens any time I try to analyze anything at all), or perhaps something dramatic has been going on in the real world. We'll look at a subset of the services with the biggest deviations to see if there's anything we can use to remove them.

In [ ]:
%%sql

with deviations as (
    select
      lineRef,
      arrivalTime,
      aimedArrivalTime,
      extract(epoch from arrivalTime - aimedArrivalTime) as delay_seconds,
    from arrivals where stopPointName = 'Nidarosdomen'
)
select *
from deviations
order by abs(delay_seconds) desc
limit 30
;

It might be hard to fix this problem. Perhaps what's happened is that the "journey" was reused, or there was a swap of which buses that would service which schedule. At first glance, this problem seems to have been bigger in the past than in the present. It's really not interesting to analyze these outliers, for now, we'll just filter out any delays longer than 30 minutes and try again.

In [ ]:
df = db.sql("""
select
  mean(extract(epoch from arrivalTime - aimedArrivalTime)) as delay_seconds,
  extract(year from arrivalTime) as year,
  extract(hour from arrivalTime) as hour,
  extract(month from arrivalTime) as month
from arrivals
where stopPointName = 'Nidarosdomen' and abs(extract(epoch from arrivalTime - aimedArrivalTime)) < 1800
group by 2, 3, 4
""").df()

sns.relplot(
    df, x='hour', col='year', col_wrap=2, y='delay_seconds', kind='line'
);

It is interesting for me to note that the schedules are optimistic on average. 0 delay does not seem to be in the 95% confidence interval of the shaded region at all. The effect of rush hour on the delay seems to have been increasing in recent years, but the variability is not huge. Let's check if some months are worse than others:

In [ ]:
sns.relplot(
    df, x='hour', col='month', col_wrap=4, y='delay_seconds', kind='line'
);

Right, that's not a big surprise -- there's much greater variability in the months when we have snowfalls. This is very visible in December, where taking the bus after work has historically been a somewhat unpredictable and annoying ordeal for me. It might not necessarily be caused by snowfalls though. Fewer people use their bike when the cold sets in, and so road traffic may also increase.

## What we learned

It seems like we can use this data set to investigate bus delays, but there are some outliers. These don't tell us much about what's _typical_ or _likely_, so we'll remove them.

We should be on the lookout for optimistic schedules. We've only investigated one stop so far, but there was a significant skew to the optimistic side. It would be interesting to check if this is caused by a few bus lines always being late, or if it distributes more uniformly.

Rush hour has an impact that the schedules for lines passing Nidarosdomen that schedules do not fully account for. We can also see that buses are less reliable when winter starts and ends. It might be interesting to pull down some weather data from [frost](https://frost.met.no/index.html) to investigate that in the future.

## Next questions

1. Are some bus lines disproportionally represented in the delays?
2. _Where_ do the buses get delayed? Is it a slow accrual of time loss due to an optimistic schedule, or are there certain areas where traffic just doesn't flow at certain times?

Let's focus on question 1 first, question 2 seems like a good way to round off this little project with some sort of fun finding that we can show. First, let's figure out which lines that visit Nidarosdomen and their mean delay. That would probably be every line going through downtown on the North-South axis, so probably a lot:

In [ ]:
%%sql
select
  lineRef,
  count(*) as count,
  mean(extract(epoch from arrivalTime - aimedArrivalTime)) as mean_delay_seconds
from arrivals
where stopPointName = 'Nidarosdomen' and abs(extract(epoch from arrivalTime - aimedArrivalTime)) < 1800
group by lineRef
order by count(*) desc
limit 30

It is easy to see that not all lines contribute the same here without any further analysis. The mean spans from more than 4 minutes to 70 seconds, and all of these have many registrations.

Let's sample a couple of lines:

- `ATB:Line:2_1` which is probably the metro bus line 1.
- `ATB:Line:2_10` which might be bus line 10?
- `ATB:Line:2_71` which might be 71 to/from Melhus?

We'll try to find out if these lines have delay distributions that have the same shape.

In [ ]:
df = db.sql("""
select
  lineRef,
  extract(epoch from arrivalTime - aimedArrivalTime) as delay_seconds,
  directionRef
from arrivals
where stopPointName = 'Nidarosdomen'
  and abs(extract(epoch from arrivalTime - aimedArrivalTime)) < 1200
  and lineRef in ('ATB:Line:2_1', 'ATB:Line:2_10', 'ATB:Line:2_71')
""").df()

sns.displot(
    df, x='delay_seconds', row='directionRef', col='lineRef', kind='hist', facet_kws={'sharey': False}, bins=30
);

The shape of all these look quite similar. Let's pick a single bus line and plot the delay by the sequenceNr to find out if it's just accruing delay gradually, or whether there's some physical place where the magic happens.

In [ ]:
df = db.sql("""
select
  lineRef,
  extract(epoch from arrivalTime - aimedArrivalTime) as delay_seconds,
  directionRef,
  sequenceNr,
  extract(year from arrivalTime) as year
from arrivals
where abs(extract(epoch from arrivalTime - aimedArrivalTime)) < 1800
  and lineRef = 'ATB:Line:2_10'
""").df()

sns.relplot(
    # This cast is a little dirty, it's just the quickest way to force seaborn
    # to create a legend for a categorical variable instead of a continuous one.
    # It'll make sure that each year gets a nice and distinct color.
    df.astype({'year': 'string'}),
    x='sequenceNr', y='delay_seconds', col='directionRef', kind='line', hue='year'
);

Wow, the error bars for these are super low, the 95% confidence interval is barely visible. I was not expecting that, it makes me believe I've done something wrong here.

I wonder if they've adjusted the schedule? At first glance, it looks like the delay is going down, possibly because of better scheduling. At least now I know that I could expect the real time estimate for this bus line to be quite accurate, even if the schedule might not be. Outbound, we're losing a few minutes after `sequenceNr` 10 up until `sequencNr` 24 or so for the year 2025. Let's take a closer look at those.

In [ ]:
%%sql
select
  stopPointName,
  sequenceNr,
  mean(extract(epoch from arrivalTime - aimedArrivalTime)) as delay_seconds
from arrivals
where
  lineRef = 'ATB:Line:2_10'
  and sequenceNr between 10 and 24
  and directionRef = 'Outbound'
  and extract(year from arrivalTime) = 2025
group by stopPointName, sequenceNr
order by sequenceNr


Ah, but these are two different, distinct schedules, so this entire section makes no sense. While these buses have the same `lineRef`, they do not start at the same origin, so it makes no sense to compare them. Let's pair up the origin and destination for this line, to find out how many different routes we would have to look at to make sensible comparisons.

In [ ]:
%%sql
select
  originName || ' - ' || destinationName as route,
  stopPointName,
  sequenceNr,
  mean(extract(epoch from arrivalTime - aimedArrivalTime)) as delay_seconds
from arrivals
where
  lineRef = 'ATB:Line:2_10'
  and sequenceNr between 1 and 5
  and directionRef = 'Outbound'
  and extract(year from arrivalTime) = 2025
group by route, stopPointName, sequenceNr
order by sequenceNr, delay_seconds

Right, so within a `lineRef`, there can be quite many routes and schedules, and the delays can be very different. Let's check the different routes within `ATB:Line:2_10` just for fun:

In [ ]:
df = db.sql("""
select
  lineRef,
  extract(epoch from arrivalTime - aimedArrivalTime) as delay_seconds,
  directionRef,
  sequenceNr,
  lineRef || ':' || originName || ' - ' || destinationName as route
from arrivals
where abs(extract(epoch from arrivalTime - aimedArrivalTime)) < 1800
  and lineRef = 'ATB:Line:2_10'
  and operatingDate >= '2025-01-01'
""").df()

sns.relplot(
    df, x='sequenceNr', y='delay_seconds', col='directionRef', kind='line', hue='route'
);

Right, there's huge variability in how reliable they are. Some run like clockwork and some have many minutes of variability. It might be interesting to look into why. Maybe some depart during rush hour only, for example? We'll leave that question for now, and look at something else.

## Analyzing journey legs

One thing that I think might be very interesting would be to check the legs of each journey to check if we can identify physical locations that cause delays according to the schedule. This seems like something we should save to a table so that we won't have to couple each stop with the next stop more than once, because this is going to be a big join. We will need to pair up each recorded arrival time at a stop, with the arrival time at the previous stop, it's going to be something like a 200 million by 200 million join.

Let's consider what we need:

1. The from-stop and to-stop. We'll get the names for convenience, and the ID so we can get geolocations later.
2. A time of day to associate with the data point so we can later check for things like rush hour or winter-time impact.
3. The planned time from arriving at the from-stop to arriving/departing at the to-stop.
4. The actual time from arriving at the from-stop to arriving/departing at the to-stop.
5. The `datedServiceJourneyId` so we can retrieve additional data by joining `arrivals` if we find out we need something else later.
6. We can probably skip `directionRef` as the from-stop/to-stop implicitly carries that information.

We will also filter out some data:

1. We will just drop journeys that have very high delays. We're interested in the typical, not the atypical.
2. Cancellations, extra stops, replacement transports, for the same reason.
3. The very first stop has no `arrivalTime` or `aimedArrivalTime` and the very last stop does not have `departureTime` or `aimedDepartureTime`. I want to check the time between arrival at the previous stop, and arrival at the next stop, including the stop itself. So we won't be able to use every row in the arrivals table.

I'm expecting this to take a few minutes, since it'll need to visit many columns of all rows and do an expensive join, so I'll make sure to only run this if the table does not already exist.

In [ ]:
exists = db.query("select exists(select 1 from information_schema.tables where table_name = 'legs')").fetchone()[0]

create = """
create table legs as select
  now.lineRef as lineRef,
  previous.stopPointName as previous_stop_name,
  previous.stopPointRef as previous_stop,
  now.stopPointName as stop_name,
  now.stopPointRef as stop,
  now.datedServiceJourneyId as datedServiceJourneyId,
  previous.arrivalTime as time,
  extract(epoch from now.aimedArrivalTime - previous.aimedArrivalTime) as planned_duration,
  extract(epoch from now.arrivalTime - previous.arrivalTime) as actual_duration,
  actual_duration - planned_duration as deviation
from arrivals previous join arrivals now on
  previous.datedServiceJourneyId = now.datedServiceJourneyId
  and previous.sequenceNr + 1 = now.sequenceNr
where
  abs(deviation) < 1800
  and not (now.journeyCancellation or now.stopCancellation or now.extraCall);
"""

if not exists:
    db.execute(create)

db.sql("select * from legs where time > '2025-01-01' limit 1").df().T

Okay, that took 3 minutes and 10 seconds on my laptop -- not too bad. **Edit**: Testing this on my desktop, which has 12 cores and 64GB RAM, it's 2 minutes. The join spills to disk on both machines, hard to avoid for such a big join.

Subsequent queries towards this table will be fast, since we don't need to do the expensive join again.

Let's check how some of this data looks, to get our bearings:

In [ ]:
%%sql

select * exclude(previous_stop, stop, datedServiceJourneyId)
from legs where stop_name = 'Nidarosdomen' and time > '2025-02-10' limit 10;

In [ ]:
%%sql
select * exclude(previous_stop, stop, datedServiceJourneyId)
from legs where stop_name = 'Berkåk' and time > '2025-02-10' limit 10;

Perfect, that looks exactly like what I'm after. In passing, I notice that the planned length is always exactly in minutes, which makes sense since these schedules are printed and physically glued to stop places. It would be unreasonable to expect second-level precision on planning them anyway. Let's check if there are any legs with a planned length of 0 seconds (which would happen if arrival at both stops is within the same minute):

In [ ]:
%%sql

select 100 * (select count(*) from legs where planned_duration = 0)
  / (select count(*) from legs) as percent

Okay, around 7% of the legs have an unreasonably tight schedule of being in two stops in the same minute. I would expect these legs to record delays most of the time, since we've defined a delay as "actual duration longer than planned duration", and we calculate "planned duration" to be 0 seconds.

In [ ]:
%%sql

select mean(deviation), median(deviation), approx_quantile(deviation, .95) from legs where planned_duration = 0

In [ ]:
%%sql

select mean(deviation), median(deviation), approx_quantile(deviation, .95) from legs where planned_duration != 0

The deviation seems typically quite low for both kinds of legs -- our scale is seconds, after all. The legs where the planned duration is 0 are probably quite short, in terms of physical distance.

We would expect this to look quite different from trains, which have further to travel between stops:

In [ ]:
%%sql

select mean(deviation), median(deviation), approx_quantile(deviation, .95) from legs where lineRef not like 'ATB:%';


An interesting thing we can easily do now is to check which legs that has the highest typical delay:

In [ ]:
%%sql

with by_stop as (
    select
      median(deviation) as deviation,
      count(*) as travels,
      previous_stop_name,
      stop_name
    from legs
    group by previous_stop_name, stop_name
    having count(*) > 10
)
select * from by_stop order by deviation desc limit 10

Lots of legs with very little traffic here, and long distances. I know that at least some of these stops are in rural areas.

Let's focus on the ones with lots of registrations, probably closer to cities.

In [ ]:
%%sql
with by_stop as (
    select
      median(deviation) as deviation,
      approx_quantile(deviation, .95) as pct_95,
      count(*) as travels,
      previous_stop_name,
      stop_name
    from legs
    group by previous_stop_name, stop_name
    having count(*) > 10000
)
select * from by_stop order by deviation desc limit 10

This is interesting stuff! I want to put this on a map visualization at some point...

A lot of these are stops that changed names, or no longer exist. For a map visualization, it would probably only be interesting with quite recent data, perhaps only the last year.

Let's take a look at only 2025 registrations.

In [ ]:
%%sql
with by_stop as (
    select
      median(deviation) as deviation,
      approx_quantile(deviation, .95) as pct_95,
      count(*) as travels,
      previous_stop_name,
      stop_name
    from legs
    where time >= '2025-01-01'
    group by previous_stop_name, stop_name
    having count(*) > 5000
)
select * from by_stop order by deviation desc limit 10

Now, I want to look into where _my_ buses get delayed. I'll look at line 14 first, it's the only line that visits my closest stop, so I'll get the `datedServiceJourneyId` from there. I want to check whether some legs typically cause much more deviations than others.

In [ ]:
%%sql

with my_bus as (
    select distinct datedServiceJourneyId from arrivals where stopPointName = 'Høyset'
), by_stop as (
    select
      median(deviation) as deviation,
      approx_quantile(deviation, .95) as pct_95,
      count(*) as travels,
      previous_stop_name,
      stop_name
    from legs join my_bus using(datedServiceJourneyId)
    group by previous_stop_name, stop_name
    having count(*) > 5000
)
select * from by_stop order by deviation desc limit 20

The median deviation is quite even. What I observe when traveling to and from work is that the roundabout between Lerkendal gård and Berg Studentby is heavily affected by rush traffic. It would be fun to try to visualize all buses that travel through it and check for rush hour traffic, focusing on the actual time taken. I want to visualize the confidence interval, which is expensive, so I'll limit this to only 2025 to make it fast.

In [ ]:
df = db.sql("""
select
  actual_duration as seconds,
  previous_stop_name || ' - ' || stop_name as from_to,
  extract(hour from time) as hour,
from legs
where
  (stop_name = 'Lerkendal gård' or previous_stop_name = 'Lerkendal gård')
  and time >= '2025-01-01' and hour between 6 and 18
""").df()

sns.relplot(
    df, x='hour', y='seconds', kind='line', hue='from_to'
);


That's fun, that closely matches what I had expected to see! It seems like rush hour makes most of these legs 30-40 seconds slower.

This makes me really want to get geolocation for stops, so I can plot this in a map and look for other hot-spots like this. Let's check how many unique previous stops and stop combinations we have, so we can get an idea of how much data an interactive visualization like that would need to collect.

In [ ]:
%%sql
select count(*) as combinations
from (select distinct previous_stop, stop from legs)

I think perhaps that's too much. What if we include only 2025?

In [ ]:
%%sql
select count(*) as combinations
from (select distinct previous_stop, stop from legs where time >= '2025-01-01')

That's still a lot of dots on a map. Let's look into whether some disappear if we only take the legs where we have over 100 registrations:

In [ ]:
%%sql
select count(*) as combinations
from (
  select previous_stop, stop, count(*)
  from legs where time >= '2025-01-01'
  group by previous_stop, stop
  having count(*) > 100
)

That seems like it could be manageable. We might be looking at a few aggregations per hour for each of these, so we'd need to load perhaps half a million data points into a browser. That should be easily manageable, especially if we can use DuckDB in the browser so we can load it from something like a parquet file or DuckDB file in object storage (S3 or GCS), which supports predicate pushdown. That would also let people play with this data on the [DuckDB Online Shell](https://shell.duckdb.org/).

## Locating stop points

I would like to be able to place stop points in a map. And I would like to be able to estimate the distance between stop points. I'll try to get all the stop points from the [national stop registry](https://data.entur.no/domain/public-transport-data/product/national_stop_registry/urn:li:container:7fd60886cfa95c660d5c85ac0384eb9a). It is my hope that I can use the IDs of the real time data set to fetch these. Let's try just one, at first.

In [ ]:
stop_point_table = "`ent-data-sharing-ext-prd.national_stop_registry.stop_places_all_versions`"

stop = db.sql("select stop from legs where stop_name = 'Nidarosdomen' limit 1").df().stop.iloc[0]
client.query(f"""
select
   id,
   version,
   name,
   location_longitude,
   location_latitude,
   validBetween
from {stop_point_table}
where id = '{stop}'
""").result().to_dataframe()

Ah, that's unfortunate. Let's check what the ID is for Nidarosdomen in this dataset and see if we can fix this:

In [ ]:
client.query(f"""
select
   id,
   version,
   name,
   location_longitude,
   location_latitude,
   validBetween
from {stop_point_table}
where name = 'Nidarosdomen'
""").result().to_dataframe()

In [ ]:
stop

Maybe what I've been calling a "stop" is actually a quay, I notice that there's another table. Let's check that:

In [ ]:
quays_table = "`ent-data-sharing-ext-prd.national_stop_registry.quays_all_versions`"

client.query(f"""
select
   id,
   version,
   name,
   location_longitude,
   location_latitude
from {quays_table}
where id = '{stop}'
""").result().to_dataframe()

Ah, seems like this is what we want, then. To simplify things, we'll accept the chance of some historical inaccuracy and use only the latest version. There's a table for that too. Let's download it. Like before, we'll run this query only if we don't already have the data.

In [ ]:
quays_table = "`ent-data-sharing-ext-prd.national_stop_registry.quays_last_version`"
exists = db.query("select exists(select 1 from information_schema.tables where table_name = 'quays')").fetchone()[0]

if not exists:
    quays = client.query(f"""
    select
       id,
       location_longitude,
       location_latitude
    from {quays_table}""").result().to_arrow()
    db.execute("create table quays as select * from quays")

db.sql("select * from quays limit 5")

Let's try to check if we miss any locations found in the real time data set.

In [ ]:
%%sql

with stops as (
    select distinct stopPointRef from arrivals
)
select count(*)
from stops left join quays on stops.stopPointRef = quays.id
where quays.id is null

Ouch, that's unfortunate. Let's take a look at a sample. Maybe we need the other table after all...

In [ ]:
%%sql
with stops as (
    select stopPointRef, stopPointName, max(operatingDate) as last_seen
    from arrivals group by all
)
select stops.stopPointRef, stops.stopPointName, stops.last_seen
from stops left join quays on stops.stopPointRef = quays.id
where quays.id is null
order by stops.last_seen desc
limit 10

It looks like maybe this has to do with recency, or trains. There are very few places we've seen used in Trondheim the past few months that we can't geolocate, plenty good enough for my purposes at least. Let's check how much of our data the past year that is impacted:

In [ ]:
%%sql
with stops as (
    select distinct stopPointRef
    from arrivals where operatingDate > '2024-01-01'
), missing as (
    select stops.stopPointRef
    from stops left join quays on stops.stopPointRef = quays.id
    where quays.id is null
), missing_arrivals as (
    select count(*)
    from arrivals join missing using(stopPointRef)
    where operatingDate > '2024-01-01'
)
select 100 * (select * from missing_arrivals)
  / (select count(*) from arrivals where operatingDate > '2024-01-01') as pct_missing

We're unable to geolocate around 2% of the data set we're looking at here, of recordings after 2024-01-01. I did a little playing around with the stop places table, looking for a few of these, and I was unable to find them. For me, it's acceptable to discard 2% of our data to be able to place the rest of it in a map.

What would be fun now, would be to estimate the distances between the stop points in our legs table. Ideally, we'd use something like driving distances, but since we also have train and possibly ferry stops, that might not be easy. I will settle for estimating a lower bound on the distance, using [haversine distance](https://en.wikipedia.org/wiki/Haversine_formula) to calculate an approximate distance as the bird flies. This will also let me do an upper bound estimate for the average speed on a leg, which might be fun to look at.

## Estimating leg distance

I find it easier to formulate the haversine distance formula in Python, so I'll pull up all the unique pairs of stops and their location into pandas, then add a distance estimate column and write it back to DuckDB. We're only looking at calculating 30k rows or so, so it should be superfast anyway.

In [ ]:
df = db.sql("""
with unique_legs as (
   select distinct previous_stop, stop from legs
)
select
    ul.previous_stop,
    prev_quay.location_longitude as prev_lon,
    prev_quay.location_latitude as prev_lat,
    ul.stop,
    cur_quay.location_longitude as now_lon,
    cur_quay.location_latitude as now_lat,
from unique_legs ul
    join quays prev_quay on ul.previous_stop = prev_quay.id
    join quays cur_quay on ul.stop = cur_quay.id
""").df()

# This is hopefully a correct haversine distance calculation, giving the air distance
# between two points on earths surface. I think this would look even more gnarly in SQL.
# We're using fast vectorized numpy operations here, so this is much faster than
# finding the distinct previous/next stops anyway
earth_radius_km = 6378.0
lon1 = np.radians(df.prev_lon)
lat1 = np.radians(df.prev_lat)
lat2 = np.radians(df.now_lat)
lon2 = np.radians(df.now_lon)

delta_lon = lon2 - lon1
delta_lat = lat2 - lat1

hav_a = np.sin(delta_lat / 2) ** 2
hav_b = np.sin(delta_lon / 2) ** 2
distance_km = earth_radius_km * 2 * np.arcsin(
    np.sqrt(hav_a + np.cos(lat1) * np.cos(lat2) * hav_b)
)
leg_distances = df.assign(
    air_distance_km=distance_km
)

db.execute(
    "create or replace table distances as select * from leg_distances"
)

leg_distances.air_distance_km.describe()

Almost all the legs have a very short air distance. This is unsurprising. Let's check how many that have a 0 distance -- that should only happen if we geolocated two stops to the same area, which is hopefully only possible if two subsequent arrivals arrived at the same `stopPointRef`.

In [ ]:
%%sql
select *
from leg_distances
where air_distance_km = 0 and previous_stop != stop

Okay, so all the 0 distances we calculated are legs were the vehicle didn't actually move anywhere. How many of our legs are like that?

In [ ]:
%%sql
select count(*) as count
from legs
where previous_stop = stop

Thankfully very few, so these aren't _actually_ a problem, we'll just discard these. I want to materialize the raw data for our legs visualization now. I'm going to write this to a new table. Expecting this to take maybe a few minutes.

In [ ]:
%%sql
create or replace table legs_distance as
select *
from legs join leg_distances using(previous_stop, stop)
where previous_stop != stop and time >= '2024-01-01'

That was actually just 10 odd seconds. Let's take a look at the fastest legs we recorded, just for fun. I certainly hope to see the trains near the top

In [ ]:
db.sql("""
with speed as (
    select *, air_distance_km / (actual_duration / 3600) as speed_km_h_lower_bound
    from legs_distance
)
select
    previous_stop_name, stop_name, time, actual_duration as duration_seconds, speed_km_h_lower_bound, air_distance_km
from speed
where actual_duration > 0
order by speed_km_h_lower_bound desc
limit 30
""").df()

Ah, looks like we found a data quality issue. Some of these bullet-buses will likely skew the map visualization, so we should note to ourselves that we should fix this.

I think we can reasonably expect all vehicles that traveled faster than 250km/h to be errors in the data. Let's check how many there are:

In [ ]:
%%sql
select 100 * mean((air_distance_km / (actual_duration / 3600) > 250) :: float) as bullet_bus_pct
from legs_distance


Okay, that's only 2% again. I feel perfectly fine with dropping those.